In [1]:
import tensorflow as tf
import os
from matplotlib import pyplot as plt
# from IPython import display
from IPython.display import clear_output

# from google.colab import drive
# drive.mount('/content/drive/')
# path = "/content/drive/My Drive/HDSP/PnP-ADMM/DATA"
# # path = "/content/drive/My Drive/"
# os.chdir(path)

from os import listdir
from os.path import isfile, join

import numpy as np
import keras
import scipy.io
import pandas as pd
from scipy.io import loadmat

from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.python.keras import Sequential
from tensorflow.keras.layers import *
from tensorflow.keras.models import *

#---------------------defined function used -------------------------------------------------------------------


# Defined protocol of input and ouput (it is recommended only read, not applied operator here, but it is possible)
def Input_image(image):
    images = loadmat(image).get('y')
    images = np.expand_dims(images,-1)
    # procesamiento
    return images/np.max(images)

def Oput_image(image):
    images = loadmat(image).get('y_m')
    images = np.expand_dims(images, -1)
    return images/np.max(images)

def load_sambles(data):
  data = data[['inimg']]
  inimg_name = list(data.iloc[:,0])
  samples = []
  for samp in inimg_name:
    samples.append(samp)
  return samples


class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, samples,PATH,batch_size=1,dim_input=(512, 512, 3), shuffle=True, dim_oput=(512, 512, 3)):
        'Initialization'
        self.dim_input = dim_input
        self.dim_oput = dim_oput
        self.batch_size = batch_size
        self.list_images = samples
        self.shuffle = shuffle
        self.PATH = PATH
        self.on_epoch_end()


    # falta __data_generation
    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(len(self.list_images) / self.batch_size)

    def __getitem__(self, index):
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index * self.batch_size:(index + 1) * self.batch_size]

        # Find list of IDs
        images_name = [self.list_images[k] for k in indexes]

        # Generate data
        X, y = self.__data_generation(images_name)

        return X, y

    def on_epoch_end(self):
        'Update indexes after each epoch'
        self.indexes = np.arange(len(self.list_images))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, images_names):
        'Generates data containing batch_size samples'  # X : (n_samples, *dim, n_channels)
        # Initialization
        X = np.empty((self.batch_size, *self.dim_input))  # Array de numpy con zeros de tamaño
        y = np.empty((self.batch_size, *self.dim_oput))
        # Generate data
        for i, file_name in enumerate(images_names):
            # Store sample
            X[i,] = Input_image(self.PATH + '/' + file_name)
            # Store class
            y[i,] = Oput_image(self.PATH + '/' + file_name)
        return X, y



def Build_data_set(IMG_WIDTH,IMG_HEIGHT,L_bands,L_imput,BATCH_SIZE,PATH,split_v):

  # Random split
  data_dir_list = os.listdir(PATH)
  N = len(data_dir_list)
  train_df = pd.DataFrame(columns=['inimg'])
  test_df = pd.DataFrame(columns=['inimg'])
  randurls = np.copy(data_dir_list)
  train_n = round(N * split_v)
  np.random.shuffle(randurls)
  tr_urls = randurls[:train_n]
  ts_urls = randurls[train_n:N]
  for i in tr_urls:
      train_df = train_df.append({'inimg': i}, ignore_index=True)
  for i in ts_urls:
      test_df = test_df.append({'inimg': i}, ignore_index=True)

  params = {'dim_input': (IMG_WIDTH, IMG_HEIGHT, L_imput),
            'dim_oput': (IMG_WIDTH, IMG_HEIGHT, L_bands),
            'batch_size': BATCH_SIZE,
            'PATH': PATH,
            'shuffle': True}

  partition_Train = load_sambles(test_df)
  partition_Test = load_sambles(train_df)

  train_generator = DataGenerator(partition_Train, **params)
  test_generator = DataGenerator(partition_Test, **params)

  train_dataset = tf.data.Dataset.from_generator(
      lambda: train_generator,
      (tf.float32, tf.float32))

  test_dataset = tf.data.Dataset.from_generator(
      lambda: test_generator,
      (tf.float32, tf.float32))

  return train_dataset,test_dataset

def PSNR_Metric(y_true, y_pred):
  return tf.reduce_mean(tf.image.psnr(y_true,y_pred,tf.reduce_max(y_true)))

os.environ["CUDA_VISIBLE_DEVICES"]= '1'
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

# gpu_info = !nvidia-smi
# gpu_info = '\n'.join(gpu_info)
# if gpu_info.find('failed') >= 0:
#   print('Not connected to a GPU')
# else:
#   print(gpu_info)
# # !pwd
# from psutil import virtual_memory
# ram_gb = virtual_memory().total / 1e9
# print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))


#----------------------------- directory of the spectral data set -------------------------                              # for windows
BATCH_SIZE = 1; L_bands    = 1; L_imput    = 1;
from scipy.io import loadmat
# from recoverynet import *     # Net build


#########################################################################################

ARAD = False # @ {type:"boolean"} ### IF TRUE MODIFIY NETWORK
if ARAD:
  IMG_WIDTH = 286; IMG_HEIGHT = 256;
  path_files  = os.getcwd()+'/ARAD/Y'
  nI = 10
else:
  IMG_WIDTH = 535; IMG_HEIGHT = 512;
  path_files  = os.getcwd()+'/REAL/Y'
  nI = 14
reTrain = False # @ {type:"boolean"}
gf=0.0  # @ {type:"raw"}
awgn=25 # @ {type:"raw"}
lr = 1e-4# @ {type:"raw"}
batch =  nI# @ {type:"raw"}

norm_inp="NO-F"  # @ {type:"raw"}
norm_out="NO-F"  # @ {type:"raw"}
loss =  'mean_squared_error'# @ {type:"string"}
model_path=os.getcwd()
net = "NO-NO"
old_cp_dir=model_path+'/Weights/Romario/' #@param {type:"raw"}
# if ARAD:
#     old_cp_dir += 'ARAD_'+str(gf)+'_'+str(awgn)+'_'+net+'/'
# else:
#     old_cp_dir += 'REAL/'




#########################################################################################

optimizad = tf.keras.optimizers.Adam(learning_rate=lr, amsgrad=False)

Y_inp = None
for s in range(nI):
  if norm_inp!="" and ARAD:
    Temp = np.array(loadmat(path_files + '/y_ideal_'+str(s+1)+'_'+norm_inp)['y_ideal'])
  else:
    Temp = np.array(loadmat(path_files + '/y_ideal_'+str(s+1))['y_ideal'])
  if Y_inp is None:
    Y_inp = np.expand_dims(Temp, 0)
  else:
    Y_inp = np.concatenate((Y_inp, np.expand_dims(Temp, 0)),0)

Y_oup = None
for s in range(nI):
  if norm_out!="" and ARAD:
    Temp = np.array(loadmat(path_files + '/y_real_m_'+str(s+1)+'_'+str(gf)+'_'+str(awgn)+'_'+norm_out)['y_real_m'])
  elif ARAD:
    Temp = np.array(loadmat(path_files + '/y_real_m_'+str(s+1)+'_'+str(gf)+'_'+str(awgn))['y_real_m'])
  else:
    Temp = np.array(loadmat(path_files + '/y_real_'+str(s+1))['y_real'])
  if Y_oup is None:
    Y_oup = np.expand_dims(Temp, 0)
  else:
    Y_oup = np.concatenate((Y_oup, np.expand_dims(Temp, 0)),0)

Y_inp = np.expand_dims(Y_inp, -1)
Y_oup = np.expand_dims(Y_oup, -1)

# clear_output()

1 Physical GPUs, 1 Logical GPUs


#UNET - Profe

In [2]:
#@title **code** a function to create the pair of UNET convolutions
def conv2d_block(input_tensor, n_filters, kernel_size = 3, batchnorm = True):
    """Function to add 2 convolutional layers with the parameters passed to it"""
    # first layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    # second layer
    x = Conv2D(filters = n_filters, kernel_size = (kernel_size, kernel_size),\
              kernel_initializer = 'he_normal', padding = 'same')(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation('relu')(x)
    
    return x
  

In [3]:
#@title **code** a nice UNET function
def get_unet(input_img, n_filters = 16, dropout = 0.1, batchnorm = True):
  # Contracting Path
  c1 = conv2d_block(input_img, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
  p1 = MaxPooling2D((2, 2))(c1)
  p1 = Dropout(dropout)(p1)
  
  c2 = conv2d_block(p1, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
  p2 = MaxPooling2D((2, 2))(c2)
  p2 = Dropout(dropout)(p2)
  
  c3 = conv2d_block(p2, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
  p3 = MaxPooling2D((2, 2))(c3)
  p3 = Dropout(dropout)(p3)
  
  c4 = conv2d_block(p3, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
  p4 = MaxPooling2D((2, 2))(c4)
  p4 = Dropout(dropout)(p4)
  
  c5 = conv2d_block(p4, n_filters = n_filters * 16, kernel_size = 3, batchnorm = batchnorm)
  
  # Expansive Path
  u6 = Conv2DTranspose(n_filters * 8, (3, 3), strides = (2, 2), padding = 'same')(c5)
  u6 = concatenate([u6, c4])
  u6 = Dropout(dropout)(u6)
  c6 = conv2d_block(u6, n_filters * 8, kernel_size = 3, batchnorm = batchnorm)
  
  u7 = Conv2DTranspose(n_filters * 4, (3, 3), strides = (2, 2), padding = 'same')(c6)
  u7 = Conv2DTranspose(n_filters * 4, [1, 2], activation='relu')(u7)
  u7 = concatenate([u7, c3])
  u7 = Dropout(dropout)(u7)
  c7 = conv2d_block(u7, n_filters * 4, kernel_size = 3, batchnorm = batchnorm)
  
  u8 = Conv2DTranspose(n_filters * 2, (3, 3), strides = (2, 2), padding = 'same')(c7)
  u8 = Conv2DTranspose(n_filters * 2, [1, 2], activation='relu')(u8)
  u8 = concatenate([u8, c2])
  u8 = Dropout(dropout)(u8)
  c8 = conv2d_block(u8, n_filters * 2, kernel_size = 3, batchnorm = batchnorm)
  
  u9 = Conv2DTranspose(n_filters * 1, (3, 3), strides = (2, 2), padding = 'same')(c8)
  u9 = Conv2DTranspose(n_filters * 1, [1, 2], activation='relu')(u9)
  u9 = concatenate([u9, c1])
  u9 = Dropout(dropout)(u9)
  c9 = conv2d_block(u9, n_filters * 1, kernel_size = 3, batchnorm = batchnorm)
  
  outputs = Conv2D(1, (1, 1), activation='sigmoid')(c9)
  # print(outputs.shape, type(outputs))
  model = Model(inputs=[input_img], outputs=[outputs])
  return model

In [4]:
import tensorflow
from keras.models import Model, load_model
from tensorflow.keras import layers
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
seed=1234 #@{type:"number"}
tf.random.set_seed(seed)
Inv_values = [True] # @param {type:"raw"}
batchnorm = True # @param {type:"boolean"}

n_filters_values = [16]#@param{type:"raw"}
dropout_values = [0.01] #@param{type:"raw"}
lr_values = [0.005] #@param{type:"raw"}



for Inv in Inv_values:
  for n_filters in n_filters_values:
    for dropout in dropout_values:
      for lr in lr_values:
        name=str(seed)+','
        if Inv:
          input_img=Y_oup
          output_img=Y_inp
          name+="-1,"
        else:
          input_img=Y_inp
          output_img=Y_oup
          name+="1,"

        input_img=np.reshape(input_img,(14,512,535,1))
        output_img=np.reshape(output_img,(14,512,535,1))

        name+= str(n_filters)+','+str(dropout)+','+str(batchnorm)+','+str(lr) # @ {type:"raw"}
        model = get_unet(Input((512,535,1)), n_filters, dropout, batchnorm)
        model.compile(optimizer=Adam(learning_rate=lr), loss="mean_squared_error", metrics=[PSNR_Metric])
        # model.summary()

        #@title **code** train the model!
        # earlystopper = EarlyStopping(patience=3, verbose=1)
        checkpointer = ModelCheckpoint(old_cp_dir+name+'.h5', verbose=1, save_best_only=True, monitor='PSNR_Metric', mode='max', save_freq='epoch')
        epochs = 100000 #@param{type:"number"}
        model.load_weights(old_cp_dir+name+'.h5')
        history = model.fit(input_img, output_img, batch_size=nI,epochs=epochs,callbacks=[checkpointer])
        last_psnr=history.history['PSNR_Metric'][-1]
        best_psnr=np.max(history.history['PSNR_Metric'])
        iter_psnr=history.history['PSNR_Metric'].index(best_psnr)
        best_loss=np.min(history.history['loss'])
        iter_loss=history.history['loss'].index(best_loss)




        name+=','+str(best_psnr)+','+str(best_loss)+','+str(epochs)+',Adam.h5'
        print(name)
        model.save(old_cp_dir+name)

        #.------------ seee the accfuracy---------------
        plt.plot(history.history['loss'], label='loss')
        plt.xlabel('Epoch')
        plt.ylabel('loss')
        plt.show()
        plt.plot(history.history['PSNR_Metric'], label='PSNR_Metric')
        plt.xlabel('Epoch')
        plt.ylabel('PSNR_Metric')
        plt.show()

Epoch 1/100000


In [ ]:
# @title print_names.py
import os
from google.colab import drive
drive.mount('/content/drive/')

def print_names():
  # folder = "/content/drive/My Drive/HDSP/PnP-ADMM/Results" #@param {type:"string"}
  folder = "/content/drive/My Drive/HDSP/PnP-ADMM/DATA/Weights/Romario" #@param {type:"string"}
  os.chdir(folder)
  for old_fn in os.listdir(folder):
    sub = ",10000," #@param {type:"string"}
    if sub in old_fn:   
      # new_fn = old_fn.replace(", ", ",").replace(sub, "") #@param {type:"raw"}
      # os.rename(folder+'/'+old_fn, folder+'/'+new_fn) 
      print(old_fn)  
print_names()

#Hide

In [ ]:
#@markdown RED-1

def UNetL(pretrained_weights=None, input_size=(256, 256, 12)):

    inputs = Input(input_size)
    # L = input_size[2];
    L = 16;
    L_2 = 2 * L;
    L_3 = 3 * L;
    L_4 = 4 * L;
    conv1 = Conv2D(L, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    conv1 = Conv2D(L, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(L_2, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = Conv2D(L_2, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(L_3, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = Conv2D(L_3, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    # drop3 = Dropout(0.5)(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(L_4, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
    conv4 = Conv2D(L_4, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    # drop4 = Dropout(0.5)(conv4)

    up5 = Conv2D(L_3, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
        UpSampling2D(size=(2, 2))(conv4))
    # up5 = Conv2DTranspose(L_3,[1,2],activation='relu')(up5)
    merge5 = concatenate([conv3, up5], axis=3)
    conv5 = Conv2D(L_3, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge5)
    conv5 = Conv2D(L_3, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)

    up6 = Conv2D(L_2, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
        UpSampling2D(size=(2, 2))(conv5))
    up6 = Conv2DTranspose(L_2, [1, 2], activation='relu')(up6)
    merge6 = concatenate([conv2, up6], axis=3)
    conv6 = Conv2D(L_2, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
    conv6 = Conv2D(L_2, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)

    up7 = Conv2D(L, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
        UpSampling2D(size=(2, 2))(conv6))
    up7 = Conv2DTranspose(L, [1, 2], activation='relu')(up7)
    merge7 = concatenate([conv1, up7], axis=3)
    conv7 = Conv2D(L, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)
    conv7 = Conv2D(L, 3, activation='relu', padding='same', kernel_initializer='he_normal', )(conv7)

    final = Conv2D(L, 1)(conv7)


    model = Model(inputs, final)

    if (pretrained_weights):
        model.load_weights(pretrained_weights)

    return model

# def UNetL2(pretrained_weights=None, input_size=(256, 256, 12)):

#     inputs = Input(input_size)
#     L = input_size[2];
#     # L = 16;
#     L_2 = 2 * L;
#     L_3 = 3 * L;
#     L_4 = 4 * L;
#     conv1 = Conv2D(L, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
#     conv1 = Conv2D(L, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
#     pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
#     conv2 = Conv2D(L_2, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
#     conv2 = Conv2D(L_2, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
#     pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
#     conv3 = Conv2D(L_3, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
#     conv3 = Conv2D(L_3, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
#     # drop3 = Dropout(0.5)(conv3)
#     pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
#     conv4 = Conv2D(L_4, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
#     conv4 = Conv2D(L_4, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
#     # drop4 = Dropout(0.5)(conv4)

#     up5 = Conv2D(L_3, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
#         UpSampling2D(size=(2, 2))(conv4))
#     up5 = Conv2DTranspose(L_3,[1,2],activation='relu')(up5)
#     merge5 = concatenate([conv3, up5], axis=3)
#     conv5 = Conv2D(L_3, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge5)
#     conv5 = Conv2D(L_3, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)

#     up6 = Conv2D(L_2, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
#         UpSampling2D(size=(2, 2))(conv5))
#     up6 = Conv2DTranspose(L_2, [1, 2], activation='relu')(up6)
#     merge6 = concatenate([conv2, up6], axis=3)
#     conv6 = Conv2D(L_2, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
#     conv6 = Conv2D(L_2, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)

#     up7 = Conv2D(L, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
#         UpSampling2D(size=(2, 2))(conv6))
#     up7 = Conv2DTranspose(L, [1, 2], activation='relu')(up7) #OFF PARA ARAD
#     merge7 = concatenate([conv1, up7], axis=3)
#     conv7 = Conv2D(L, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)
#     conv7 = Conv2D(L, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv7)

#     final = Conv2D(1, 1)(conv7)


#     model = Model(inputs, final)

#     if (pretrained_weights):
#         model.load_weights(pretrained_weights)

#     return model

In [ ]:
#@markdown RED-2
def UNetL2_REAL(pretrained_weights=None, input_size=(256, 256, 12)):
    inputs = Input(input_size)
    L = 16;
    L_2 = 2 * L;
    L_3 = 3 * L;
    L_4 = 4 * L;
    conv1 = Conv2D(L, 3, activation='relu', padding='same', kernel_initializer='he_normal')(inputs)
    
    conv1 = Conv2D(L, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(L_2, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool1)
    conv2 = Conv2D(L_2, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(L_3, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool2)
    conv3 = Conv2D(L_3, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv3)
    # drop3 = Dropout(0.5)(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(L_4, 3, activation='relu', padding='same', kernel_initializer='he_normal')(pool3)
    conv4 = Conv2D(L_4, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv4)
    # drop4 = Dropout(0.5)(conv4)

    up5 = Conv2D(L_3, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
        UpSampling2D(size=(2, 2))(conv4))
    up5 = Conv2DTranspose(L_3, [1, 2], activation='relu')(up5)
    merge5 = concatenate([conv3, up5], axis=3)
    conv5 = Conv2D(L_3, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge5)
    conv5 = Conv2D(L_3, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv5)

    up6 = Conv2D(L_2, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
        UpSampling2D(size=(2, 2))(conv5))
    up6 = Conv2DTranspose(L_2, [1, 2], activation='relu')(up6)
    merge6 = concatenate([conv2, up6], axis=3)
    conv6 = Conv2D(L_2, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge6)
    conv6 = Conv2D(L_2, 3, activation='relu', padding='same', kernel_initializer='he_normal')(conv6)

    up7 = Conv2D(L, 2, activation='relu', padding='same', kernel_initializer='he_normal')(
        UpSampling2D(size=(2, 2))(conv6))
    up7 = Conv2DTranspose(L, [1, 2], activation='relu')(up7)
    merge7 = concatenate([conv1, up7], axis=3)
    conv7 = Conv2D(L, 3, activation='relu', padding='same', kernel_initializer='he_normal')(merge7)
    conv7 = Conv2D(L, 3, activation='relu', padding='same', kernel_initializer='he_normal', )(conv7)

    final = Conv2D(1, 1)(conv7)

    model = Model(inputs, final)

    if (pretrained_weights):
        model.load_weights(pretrained_weights)
    return model

In [ ]:
input_img = tf.keras.layers.Input(shape=(512, 535, 1) ) 
x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(input_img)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
encoded = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x) 

x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(128, [1, 2], activation='relu')(x)
decoded = tf.keras.layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder_cnn = tf.keras.models.Model(input_img, decoded)

opt = tf.optimizers.Adam(learning_rate = 0.01)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=old_cp_dir+'bestCAE.h5',
    save_weights_only=True,
    verbose=1,
    save_freq='epoch',
    monitor='PSNR_Metric',
    mode='max',
    save_best_only=True)

epochs =  1000# @param {type:"number"}
autoencoder_cnn.compile(optimizer=opt, loss=loss, metrics = [PSNR_Metric])
autoencoder_cnn.summary()

In [ ]:
input_img = tf.keras.layers.Input(shape=(512, 535, 1) ) 
x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(input_img)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
encoded = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x) 

x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(128, [1, 2], activation='relu')(x)
decoded = tf.keras.layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder_cnn = tf.keras.models.Model(input_img, decoded)

opt = tf.optimizers.Adam(learning_rate = 0.01)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=old_cp_dir+'bestCAE.h5',
    save_weights_only=True,
    verbose=1,
    save_freq='epoch',
    monitor='PSNR_Metric',
    mode='max',
    save_best_only=True)

epochs =  10000# @param {type:"number"}
autoencoder_cnn.compile(optimizer=opt, loss=loss, metrics = [PSNR_Metric])
autoencoder_cnn.summary()
# history = autoencoder_cnn.fit(Y_inp,Y_oup, callbacks=[model_checkpoint_callback], epochs=epochs)

# last_psnr=history.history['PSNR_Metric'][-1]
# best_psnr=np.max(history.history['PSNR_Metric'])
# iter_psnr=history.history['PSNR_Metric'].index(best_psnr)
# best_loss=np.min(history.history['loss'])
# iter_loss=history.history['loss'].index(best_loss)
# name="AutoencoderCNN" # @param {type:"string"}
# name+=', '+str(best_psnr)+', '+str(best_loss)+', '+str(epochs)+'.h5'
# print(name)
# autoencoder_cnn.save(old_cp_dir+name)

# #.------------ seee the accfuracy---------------
# plt.plot(history.history['loss'], label='loss')
# plt.xlabel('Epoch')
# plt.ylabel('loss')
# plt.show()
# plt.plot(history.history['PSNR_Metric'], label='PSNR_Metric')
# plt.xlabel('Epoch')
# plt.ylabel('PSNR_Metric')
# plt.show()

In [ ]:
input_img = tf.keras.layers.Input(shape=(512, 535, 1) ) 
x = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(input_img)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same')(x)
encoded = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x) 

x = tf.keras.layers.Conv2D(8, (3, 3), activation='relu', padding='same')(encoded)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(32, [1, 2], activation='relu')(x)
x = tf.keras.layers.Conv2D(16, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
# x = tf.keras.layers.Conv2D(32, [1, 3], activation='relu')(x)
x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(32, [1, 2], activation='relu')(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.UpSampling2D((2, 2))(x)

x = tf.keras.layers.Conv2D(256, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(256, [1, 2], activation='relu')(x)
decoded = tf.keras.layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder_cnn = tf.keras.models.Model(input_img, decoded)

opt = tf.optimizers.Adam(learning_rate = 0.001)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=old_cp_dir+'bestCAE.h5',
    save_weights_only=True,
    verbose=1,
    save_freq='epoch',
    monitor='PSNR_Metric',
    mode='max',
    save_best_only=True)
#@markdown CAE
autoencoder_cnn.compile(optimizer=opt, loss=loss, metrics = [PSNR_Metric])
autoencoder_cnn.summary()

In [ ]:
epochs = 1000 #@param{type:"number"}
history = autoencoder_cnn.fit(Y_inp,Y_oup, callbacks=[model_checkpoint_callback], epochs=epochs)

last_psnr=history.history['PSNR_Metric'][-1]
best_psnr=np.max(history.history['PSNR_Metric'])
iter_psnr=history.history['PSNR_Metric'].index(best_psnr)
best_loss=np.min(history.history['loss'])
iter_loss=history.history['loss'].index(best_loss)
name="CAE2" # @param {type:"string"}
name+=', '+str(best_psnr)+', '+str(best_loss)+', '+str(epochs)+'.h5'
print(name)
autoencoder_cnn.save(old_cp_dir+name)

#.------------ seee the accfuracy---------------
plt.plot(history.history['loss'], label='loss')
plt.xlabel('Epoch')
plt.ylabel('loss')
plt.show()
plt.plot(history.history['PSNR_Metric'], label='PSNR_Metric')
plt.xlabel('Epoch')
plt.ylabel('PSNR_Metric')
plt.show()

In [ ]:
input_img = tf.keras.layers.Input(shape=(512, 535, 1) ) 
x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(input_img)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x)
x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(x)
encoded = tf.keras.layers.MaxPooling2D((2, 2), padding='same')(x) 

x = tf.keras.layers.Conv2D(32, (3, 3), activation='relu', padding='same')(encoded)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(64, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(128, (3, 3), activation='relu', padding='same')(x)
x = tf.keras.layers.UpSampling2D((2, 2))(x)
x = tf.keras.layers.Conv2D(128, [1, 2], activation='relu')(x)
decoded = tf.keras.layers.Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

autoencoder_cnn = tf.keras.models.Model(input_img, decoded)

opt = tf.optimizers.Adam(learning_rate = 0.01)

model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=old_cp_dir+'bestCAE.h5',
    save_weights_only=True,
    verbose=1,
    save_freq='epoch',
    monitor='PSNR_Metric',
    mode='max',
    save_best_only=True)

epochs =  10000# @param {type:"number"}
autoencoder_cnn.compile(optimizer=opt, loss=loss, metrics = [PSNR_Metric])
history = autoencoder_cnn.fit(Y_inp,Y_oup, callbacks=[model_checkpoint_callback], epochs=epochs)

last_psnr=history.history['PSNR_Metric'][-1]
best_psnr=np.max(history.history['PSNR_Metric'])
iter_psnr=history.history['PSNR_Metric'].index(best_psnr)
best_loss=np.min(history.history['loss'])
iter_loss=history.history['loss'].index(best_loss)
name="AutoencoderCNN" # @param {type:"string"}
name+=', '+str(best_psnr)+', '+str(best_loss)+', '+str(epochs)+'.h5'
print(name)
autoencoder_cnn.save(old_cp_dir+name)

#.------------ seee the accfuracy---------------
plt.plot(history.history['loss'], label='loss')
plt.xlabel('Epoch')
plt.ylabel('loss')
plt.show()
plt.plot(history.history['PSNR_Metric'], label='PSNR_Metric')
plt.xlabel('Epoch')
plt.ylabel('PSNR_Metric')
plt.show()

In [ ]:

epochs =  10000# @param {type:"number"}
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=old_cp_dir+'best.h5',
    save_weights_only=True,
    verbose=1,
    save_freq='epoch',
    monitor='PSNR_Metric',
    mode='max',
    save_best_only=True)

#-------------Net_model----------------------------------------------------------------
model = UNetL2_REAL(input_size=(IMG_HEIGHT,IMG_WIDTH,L_bands))
# model = UNetL(input_size=(256,256+27-1,1))
model.load_weights('model_New_data_v10.h5')
# model.load_weights('model_weights_normal_nv3.h5')
model.compile(optimizer=optimizad, loss=loss, metrics = [PSNR_Metric])
history = model.fit(x=Y_inp,y=Y_oup, epochs=epochs, callbacks=[model_checkpoint_callback])
last_psnr=history.history['PSNR_Metric'][-1]
best_psnr=np.max(history.history['PSNR_Metric'])
iter_psnr=history.history['PSNR_Metric'].index(best_psnr)
best_loss=np.min(history.history['loss'])
iter_loss=history.history['loss'].index(best_loss)
name="Default" # @param {type:"string"}
name+=', '+str(best_psnr)+', '+str(best_loss)+', '+str(epochs)+'.h5'
print(name)
model.save(old_cp_dir+name)

#.------------ seee the accfuracy---------------
plt.plot(history.history['loss'], label='loss')
plt.xlabel('Epoch')
plt.ylabel('loss')
plt.show()
plt.plot(history.history['PSNR_Metric'], label='PSNR_Metric')
plt.xlabel('Epoch')
plt.ylabel('PSNR_Metric')
plt.show()

# Ruben

In [ ]:
#@markdown UNet - Variance Scaling - Padd - Batch
def UNetL2(pretrained_weights=None, input_size=(256, 256, 12)):

    init = tf.keras.initializers.VarianceScaling(
      scale=1.0,
      mode='fan_in',
      distribution='truncated_normal',
      seed=None
    )
    inputs = Input(input_size)
    L = 16;
    L_2 = 2 * L;
    L_3 = 3 * L;
    L_4 = 4 * L;
    conv1 = Conv2D(L, 3, activation='relu', padding='same', kernel_initializer=init)(inputs)
    conv1 = Conv2D(L, 3, activation='relu', padding='same', kernel_initializer=init)(conv1)
    conv1 = BatchNormalization()(conv1)
    pool1 = MaxPooling2D(pool_size=(2, 2))(conv1)
    conv2 = Conv2D(L_2, 3, activation='relu', padding='same', kernel_initializer=init)(pool1)
    conv2 = Conv2D(L_2, 3, activation='relu', padding='same', kernel_initializer=init)(conv2)
    conv2 = BatchNormalization()(conv2)
    pool2 = MaxPooling2D(pool_size=(2, 2))(conv2)
    conv3 = Conv2D(L_3, 3, activation='relu', padding='same', kernel_initializer=init)(pool2)
    conv3 = Conv2D(L_3, 3, activation='relu', padding='same', kernel_initializer=init)(conv3)
    conv3 = BatchNormalization()(conv3)
    pool3 = MaxPooling2D(pool_size=(2, 2))(conv3)
    conv4 = Conv2D(L_4, 3, activation='relu', padding='same', kernel_initializer=init)(pool3)
    conv4 = Conv2D(L_4, 3, activation='relu', padding='same', kernel_initializer=init)(conv4)
    conv4 = BatchNormalization()(conv4)
    up5 = Conv2D(L_3, 2, activation='relu', padding='same', kernel_initializer=init)(
        UpSampling2D(size=(2, 2))(conv4))
    up5 = Conv2DTranspose(L_3,[1,2],activation='relu')(up5)
    merge5 = concatenate([conv3, up5], axis=3)
    conv5 = Conv2D(L_3, 3, activation='relu', padding='same', kernel_initializer=init)(merge5)
    conv5 = Conv2D(L_3, 3, activation='relu', padding='same', kernel_initializer=init)(conv5)

    conv5 = BatchNormalization()(conv5)
    up6 = Conv2D(L_2, 2, activation='relu', padding='same', kernel_initializer=init)(
        UpSampling2D(size=(2, 2))(conv5))
    up6 = Conv2DTranspose(L_2, [1, 2], activation='relu')(up6)
    merge6 = concatenate([conv2, up6], axis=3)
    conv6 = Conv2D(L_2, 3, activation='relu', padding='same', kernel_initializer=init)(merge6)
    conv6 = Conv2D(L_2, 3, activation='relu', padding='same', kernel_initializer=init)(conv6)

    conv6 = BatchNormalization()(conv6)
    up7 = Conv2D(L, 2, activation='relu', padding='same', kernel_initializer=init)(
        UpSampling2D(size=(2, 2))(conv6))
    up7 = Conv2DTranspose(L, [1, 2], activation='relu')(up7) #OFF PARA ARAD
    merge7 = concatenate([conv1, up7], axis=3)
    conv7 = Conv2D(L, 3, activation='relu', padding='same', kernel_initializer=init)(merge7)
    conv7 = Conv2D(L, 3, activation='relu', padding='same', kernel_initializer=init, )(conv7)

    final = Conv2D(1, 1)(conv7)


    model = Model(inputs, final)

    if (pretrained_weights):
        model.load_weights(pretrained_weights)

    return model

In [ ]:
# tf.random.set_seed(1234)
epochs =  1000# @param {type:"number"}
name="Inv_VarianceScaling_Padd_Batch_seed_1234" # @param {type:"string"}
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=old_cp_dir+name+'.h5',
    save_weights_only=True,
    verbose=1,
    save_freq='epoch',
    monitor='PSNR_Metric',
    mode='max',
    save_best_only=True)

#-------------Net_model----------------------------------------------------------------
model = UNetL2(input_size=(IMG_HEIGHT,IMG_WIDTH,L_bands))
model.compile(optimizer=optimizad, loss=loss,metrics = [PSNR_Metric])
history = model.fit(x=Y_oup,y=Y_inp, epochs=epochs, callbacks=[model_checkpoint_callback])
last_psnr=history.history['PSNR_Metric'][-1]
best_psnr=np.max(history.history['PSNR_Metric'])
iter_psnr=history.history['PSNR_Metric'].index(best_psnr)
best_loss=np.min(history.history['loss'])
iter_loss=history.history['loss'].index(best_loss)




name+=', '+str(best_psnr)+', '+str(best_loss)+', '+str(epochs)+'.h5'
print(name)
model.save(old_cp_dir+name)

#.------------ seee the accfuracy---------------
plt.plot(history.history['loss'], label='loss')
plt.xlabel('Epoch')
plt.ylabel('loss')
plt.show()
plt.plot(history.history['PSNR_Metric'], label='PSNR_Metric')
plt.xlabel('Epoch')
plt.ylabel('PSNR_Metric')
plt.show()